# Lesson2 畳み込みニューラルネットワーク (CNN)

## 目次

- Section1 解説
  - 1.1 CNN基礎
  - 1.2 Convolution(畳み込み)層
  - 1.3 Pooling(プーリング)層
  - 1.4 確認問題
- Section2 実装①
  - 2.1 Fasion MNISTをCNNでクラス分類
  - 2.2 CIFAR10のデータをCNNでクラス分類
- Section3 テクニック・発展内容
  - 3.1 Data Augmentation
  - 3.2 画像データの正規化
  - 3.3 Batch Normalization
  - 3.4 Skip Connection  (Residual Network)
  - 3.5 学習済みネットワークの利用
  - 3.6 学習させたモデルの保存・再利用
  - 3.7 確認問題
- Section4 実装②
  - 4.1 CIFAR10のデータをCNNでクラス分類②
- Section5 ケーススタディ

## 3.7 の解答

問1: ② 問2: ① 問3: ① 問4: ①

In [1]:
%matplotlib inline

import os

import tensorflow.python.keras as keras
import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input, Activation, add, Add, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.models import Sequential, Model
from keras.datasets import cifar10
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from IPython.display import SVG
from tensorflow.python.keras.utils.vis_utils import model_to_dot
from keras.preprocessing.image import ImageDataGenerator

random_state = 42

2024-07-29 22:16:50.820911: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Section4 実装②

### 4.1 CIFAR10のデータをCNNでクラス分類②

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32') / 255
y_train = np.eye(10)[y_train.astype('int32').flatten()]

x_test = x_test.astype('float32') / 255
y_test = np.eye(10)[y_test.astype('int32').flatten()]

x_train, x_valid, y_train, y_valid = train_test_split(
    x_train, y_train, test_size=10000)

Section3の学習内容も踏まえて、CIFAR10のクラス分類を行いたいと思います。

まず、モデルの作成を行います。

In [6]:
model = Sequential()

model.add(Conv2D(6, kernel_size=(5, 5), activation='relu',
                 kernel_initializer='he_normal', input_shape=(32, 32, 3)))  # 32x32x3 -> 28x28x6
model.add(MaxPooling2D(pool_size=(2, 2)))  # 28x28x6 -> 14x14x6
model.add(Conv2D(16, kernel_size=(5, 5), activation='relu',
                 kernel_initializer='he_normal'))  # 14x14x6 -> 10x10x16
model.add(MaxPooling2D(pool_size=(2, 2)))  # 10x10x16 -> 5x5x16

model.add(Flatten())  # 5x5x16 -> 400
model.add(Dense(120, activation='relu',
                kernel_initializer='he_normal'))  # 400 ->120
model.add(Dense(84, activation='relu', kernel_initializer='he_normal'))  # 120 ->84
model.add(Dense(10, activation='softmax'))  # 84 ->10

model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)

2024-07-29 22:38:58.420409: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


次に、Section3で学習したDataAugumentationや画像データの正規化を学習に反映させてみます。

kerasのImageDataGeneratorを使用します。

In [7]:
datagen = ImageDataGenerator(
    width_shift_range=0.2,  # 3.1.1 左右にずらす
    height_shift_range=0.2,  # 3.1.2 上下にずらす
    horizontal_flip=True,  # 3.1.3 左右反転
    # 3.2.1 Global Contrast Normalization (GCN) (Falseに設定しているのでここでは使用していない)
    samplewise_center=False,
    samplewise_std_normalization=False,
    zca_whitening=False)  # 3.2.2 Zero-phase Component Analysis (ZCA) Whitening (Falseに設定しているのでここでは使用していない)

In [8]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=100),
                    steps_per_epoch=x_train.shape[0] // 100, epochs=30, validation_data=(x_valid, y_valid))

Epoch 1/30


/var/folders/71/3hjq04pn7ddfr_2j2g5f52c00000gn/T/ipykernel_36024/3786900849.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(x_train, y_train, batch_size=100),


400/400 [==============================] - 17s 42ms/step - loss: 1.8623 - accuracy: 0.3176 - val_loss: 1.5899 - val_accuracy: 0.4258
Epoch 2/30
400/400 [==============================] - 16s 41ms/step - loss: 1.6560 - accuracy: 0.3948 - val_loss: 1.5239 - val_accuracy: 0.4553
Epoch 3/30
400/400 [==============================] - 16s 40ms/step - loss: 1.5770 - accuracy: 0.4272 - val_loss: 1.4667 - val_accuracy: 0.4764
Epoch 4/30
400/400 [==============================] - 16s 41ms/step - loss: 1.5167 - accuracy: 0.4534 - val_loss: 1.3876 - val_accuracy: 0.5019
Epoch 5/30
400/400 [==============================] - 16s 41ms/step - loss: 1.4771 - accuracy: 0.4664 - val_loss: 1.3001 - val_accuracy: 0.5410
Epoch 6/30
400/400 [==============================] - 16s 40ms/step - loss: 1.4473 - accuracy: 0.4777 - val_loss: 1.3184 - val_accuracy: 0.5280
Epoch 7/30
400/400 [==============================] - 16s 40ms/step - loss: 1.4167 - accuracy: 0.4911 - val_loss: 1.2743 - val_accuracy: 0.5446
Epo